<a href="https://colab.research.google.com/github/rdinamar/datos_sigep_sernameg/blob/main/datos_sigep_generico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamiento datos Sigep SernamEG

El siguiente código, desarrollado en Python, facilita la consolidación de los diferentes talleres y jornadas ejecutados a nivel regional.

El programa cuenta con dos bloques:
* Carga de los 13 archivos descargados desde sigep (cada unos de los archivos que entrega Sigep)
* Procesamiento de datos, para obtener un Excel que contiene la totalidad de los talleres y jornadas en la región.

Además el excel de salida entrega variables adicionales:
* Cuatrimestre como variable categórica.
* Avance: que permite ver deficit o superavit del taller o jornada.
* Fecha de realización del reporte.
* Max_fecha última fecha registrada en el taller o jornada.
* mes: número de mes que corresponde la fecha de inicio del taller o jornada

Y por último crea un archivo Excel en donde esta un resumen de la región con cada una de las provincia.

Se crea una tabla con la variable avance de cada uno de los talleres y jornadas, facilitando la visualización del avance programatico. Los números negativos significa deficit y positivos superavit.

Y un hoja de resumen que calcula las cobertura programada, mujeres activas y porcentaje de avance entre programadas y activas.

Para dudas y consultas pueden escribir a mi correo rdinamarca@prodemu.cl

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Sistema de Gestión y Planificación 2025 (4).xlsx to Sistema de Gestión y Planificación 2025 (4).xlsx
Saving Sistema de Gestión y Planificación 2025 (5).xlsx to Sistema de Gestión y Planificación 2025 (5).xlsx
Saving Sistema de Gestión y Planificación 2025 (6).xlsx to Sistema de Gestión y Planificación 2025 (6).xlsx
Saving Sistema de Gestión y Planificación 2025 (7).xlsx to Sistema de Gestión y Planificación 2025 (7).xlsx
Saving Sistema de Gestión y Planificación 2025 (8).xlsx to Sistema de Gestión y Planificación 2025 (8).xlsx
Saving Sistema de Gestión y Planificación 2025 (9).xlsx to Sistema de Gestión y Planificación 2025 (9).xlsx
Saving Sistema de Gestión y Planificación 2025 (10).xlsx to Sistema de Gestión y Planificación 2025 (10).xlsx
Saving Sistema de Gestión y Planificación 2025 (11).xlsx to Sistema de Gestión y Planificación 2025 (11).xlsx
Saving Sistema de Gestión y Planificación 2025 (12).xlsx to Sistema de Gestión y Planificación 2025 (12).xlsx
Saving Sistema de Gest

In [2]:
import pandas as pd
import glob
import numpy as np
from datetime import datetime

# === LECTURA DE ARCHIVOS ===
ruta_carpeta = "/content"  # Cambia por tu ruta
archivos = glob.glob(f"{ruta_carpeta}/*.xlsx")

dataframes = []
for archivo in archivos:
    try:
        df_temp = pd.read_excel(archivo, skiprows=1)
        dataframes.append(df_temp)
        print(f"Archivo leído: {archivo}")
    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

df = pd.concat(dataframes, ignore_index=False)

# === LIMPIEZA Y VARIABLES ===
df['Fecha Inicio'] = pd.to_datetime(df['Fecha Inicio'], format='%d/%m/%Y', errors='coerce')
df['Fecha Termino'] = pd.to_datetime(df['Fecha Termino'], format='%d/%m/%Y', errors='coerce')
df['Fecha Termino.1'] = pd.to_datetime(df['Fecha Termino.1'], format='%d/%m/%Y', errors='coerce')
df['Fecha Termino.2'] = pd.to_datetime(df['Fecha Termino.2'], format='%d/%m/%Y', errors='coerce')

df["Cuatrimestre"] = np.select(
    condlist=[
        (df["Fecha Inicio"] >= '2025-01-01') & (df["Fecha Inicio"] <= '2025-04-30'),
        (df["Fecha Inicio"] >= '2025-05-01') & (df["Fecha Inicio"] <= '2025-08-31'),
        (df["Fecha Inicio"] >= '2025-09-01') & (df["Fecha Inicio"] <= '2025-12-30')
    ],
    choicelist=["Q1", "Q2", "Q3"],
    default="Error de tipeo"
)

df['Avance'] = (df['N° de participantes programadas'] * -1) + df['N° de participantes activas']
df['Fecha_informe'] = datetime.now().strftime('%Y-%m-%d')
df['max_fecha'] = df[['Fecha Termino', 'Fecha Termino.1', 'Fecha Termino.2']].max(axis=1, skipna=True)
df['mes'] = df['Fecha Inicio'].dt.strftime('%m')

# === TABLA REGIONAL ===
q_region = pd.crosstab(df['Formato'], df['Cuatrimestre'], values=df['Avance'], aggfunc='sum')

# === ESTADÍSTICAS REGIONALES ===
sum_n_prog = df['N° de participantes programadas'].sum()
sum_n_act = df['N° de participantes activas'].sum()
p_prog = round(sum_n_act / sum_n_prog * 100, 2) if sum_n_prog > 0 else 0

resumen_region = {
    "texto_total_prog": f"El total de participantes programadas es {sum_n_prog}",
    "texto_total_activas": f"El total de participantes activas es {sum_n_act}",
    "texto_porcentaje": f"El porcentaje de participantes activas es {p_prog}"
}

# === PROCESAMIENTO AUTOMÁTICO POR PROVINCIA ===
provincias = df['Provincia'].dropna().unique()
resultados_provincias = {}  # para almacenar tablas y textos

for prov in provincias:
    df_prov = df[df['Provincia'] == prov]

    q_prov = pd.crosstab(df_prov['Formato'], df_prov['Cuatrimestre'], values=df_prov['Avance'], aggfunc='sum')

    sum_prog = df_prov['N° de participantes programadas'].sum()
    sum_act = df_prov['N° de participantes activas'].sum()
    p_act = round(sum_act / sum_prog * 100, 2) if sum_prog > 0 else 0

    resultados_provincias[prov] = {
        "tabla": q_prov,
        "texto_prog": f"El total de participantes programadas es {sum_prog}",
        "texto_activas": f"El total de participantes activas es {sum_act}",
        "texto_porcentaje": f"El porcentaje de participantes activas es {p_act}"
    }

# === EXPORTACIÓN ===
fecha_actual = datetime.now().strftime("%Y-%m-%d")
nombre_archivo = f"{ruta_carpeta}/completo_{fecha_actual}.xlsx"

with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
    # Hoja general
    df.to_excel(writer, sheet_name='talleres', index=False)
    q_region.to_excel(writer, sheet_name='q_region')

    # Escribir resumen regional
    resumen_df = pd.DataFrame({
        "Indicador": ["Programadas", "Activas", "Porcentaje"],
        "Valor": [sum_n_prog, sum_n_act, f"{p_prog}%"]
    })
    resumen_df.to_excel(writer, sheet_name='resumen_region', index=False)

    # Provincias dinámicas
    for prov, datos in resultados_provincias.items():
        hoja = prov.replace(" ", "_")  # evita errores con espacios
        datos["tabla"].to_excel(writer, sheet_name=f"q_{hoja}")

        resumen_prov_df = pd.DataFrame({
            "Indicador": ["Programadas", "Activas", "Porcentaje"],
            "Valor": [
                datos["texto_prog"].split()[-1],
                datos["texto_activas"].split()[-1],
                datos["texto_porcentaje"].split()[-1]
            ]
        })
        resumen_prov_df.to_excel(writer, sheet_name=f"resumen_{hoja}", index=False)

print(f"\n✅ Archivo generado correctamente: {nombre_archivo}")



Archivo leído: /content/Sistema de Gestión y Planificación 2025 (11).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (5).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (4).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (8).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (9).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (13).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (14).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (6).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (7).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (10).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (12).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (16).xlsx
Archivo leído: /content/Sistema de Gestión y Planificación 2025 (15).xlsx

✅ Archivo generado correctamente: /content/